# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col='City_ID')

# Display sample data
city_data_df.head()

,City,Latitude,Longitude,Max Temperature (C),Humidity (%),Cloudiness (%),Wind Speed (m/sec),Country,Date
City_ID,,,,,,,,,
0,longyearbyen,78.2186,15.6401,-3.09,86,75,3.09,SJ,1698523990
1,bredasdorp,-34.5322,20.0403,14.05,46,100,6.81,ZA,1698523880
2,praia,14.9215,-23.5087,26.30,73,20,6.69,CV,1698523990
3,atafu village,-8.5421,-172.5159,28.38,74,39,4.21,TK,1698523990
4,puerto natales,-51.7236,-72.4875,3.25,87,75,6.69,CL,1698523990


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Longitude',
    y='Latitude',
    geo=True,
    size='Humidity (%)',
    tiles='OSM',
    c='City',
    cmap='tab20',
    height=800,
    width=1200,
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit defined criteria and drop any results with null values
condition_temp_high = city_data_df['Max Temperature (C)'] <= 28
condition_temp_low = city_data_df['Max Temperature (C)'] >= 24
condition_hum_high = city_data_df['Humidity (%)'] <= 70
condition_hum_low = city_data_df['Humidity (%)'] >= 30
condition_country_bad = ['AF', 'BY', 'BF','CF', 'MM', 'PS', 'HT', 'IR', 'IQ', 'LB', 'LY', 'ML', 'MX', 'NE', 'KP', 'PK','RU', 'SO', 'SS', 'SD', 'SY', 'UA', 'VE', 'YE']

best_cities_df = city_data_df[condition_temp_high & condition_temp_low & condition_hum_high & condition_hum_low & (~city_data_df['Country'].isin(condition_country_bad))]

# Drop any rows with null values
best_cities_df_cleaned = best_cities_df.dropna()

# Display sample data
best_cities_df_cleaned.head()

,City,Latitude,Longitude,Max Temperature (C),Humidity (%),Cloudiness (%),Wind Speed (m/sec),Country,Date
City_ID,,,,,,,,,
50,lompoc,34.6391,-120.4579,25.01,63,0,8.75,US,1698523997
54,isla vista,34.4133,-119.8610,26.91,50,0,2.06,US,1698523998
55,bethel,41.3712,-73.4140,26.70,62,75,4.12,US,1698523998
63,puerto baquerizo moreno,-0.9000,-89.6000,27.55,69,75,6.69,EC,1698523833
126,avarua,-21.2078,-159.7750,24.03,64,75,4.63,CK,1698524011


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_cities_df_cleaned[['City', 'Country', 'Latitude', 'Longitude', 'Humidity (%)']].reset_index()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:, 'Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City_ID,City,Country,Latitude,Longitude,Humidity (%),Hotel Name
0,50,lompoc,US,34.6391,-120.4579,63,
1,54,isla vista,US,34.4133,-119.8610,50,
2,55,bethel,US,41.3712,-73.4140,62,
3,63,puerto baquerizo moreno,EC,-0.9000,-89.6000,69,
4,126,avarua,CK,-21.2078,-159.7750,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    coordinates_lat = hotel_df.loc[index, "Latitude"]
    coordinates_long = hotel_df.loc[index, "Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{coordinates_long},{coordinates_lat},{radius}"
    params["bias"] = f"proximity:{coordinates_long},{coordinates_lat}"
    params["categories"] = f"accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
isla vista - nearest hotel: The Club & Guest House
bethel - nearest hotel: Hampton Inn Danbury
puerto baquerizo moreno - nearest hotel: Dory's House
avarua - nearest hotel: Paradise Inn
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
sao jose da coroa grande - nearest hotel: Hotel do Sol
karratha - nearest hotel: Karratha International Hotel
hurghada - nearest hotel: Sea Horse Hotel
touros - nearest hotel: Pousada Atlântico
kapa'a - nearest hotel: Pono Kai Resort
acarau - nearest hotel: No hotel found
tarariras - nearest hotel: No hotel found
nadi - nearest hotel: President Hotel
arambol - nearest hotel: El Paso
santa ana - nearest hotel: Holiday Inn Express
wamba - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
cabedelo - nearest hotel: No hotel found
al hufuf - nearest hotel: Al Muhaidab Residence Al Ahsa
lihue - nearest hotel: Kauai Palms
save - nearest hot

,City_ID,City,Country,Latitude,Longitude,Humidity (%),Hotel Name
0,50,lompoc,US,34.6391,-120.4579,63,Embassy Suites by Hilton Lompoc Central Coast
1,54,isla vista,US,34.4133,-119.8610,50,The Club & Guest House
2,55,bethel,US,41.3712,-73.4140,62,Hampton Inn Danbury
3,63,puerto baquerizo moreno,EC,-0.9000,-89.6000,69,Dory's House
4,126,avarua,CK,-21.2078,-159.7750,64,Paradise Inn
5,133,wailua homesteads,US,22.0669,-159.3780,66,Hilton Garden Inn Kauai Wailua Bay
6,149,sao jose da coroa grande,BR,-8.8978,-35.1478,68,Hotel do Sol
7,166,karratha,AU,-20.7377,116.8463,69,Karratha International Hotel
8,171,hurghada,EG,27.2574,33.8129,65,Sea Horse Hotel
9,200,touros,BR,-5.1989,-35.4608,69,Pousada Atlântico


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Longitude',
    y='Latitude',
    geo=True,
    size='Humidity (%)',
    tiles='OSM',
    c='City',
    cmap='tab20',
    height=800,
    width=1200,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%),Hotel Name,Country)